In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0";  
from torchvision import models
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
import torch.optim as optim
import numpy as np
import mmd
import numpy as np
import random
from scipy.io import loadmat, savemat
from datetime import datetime
# dir(models)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # if not use cpu
print(device)


    
class create_model(nn.Module):
    def __init__(self,feature_Size,num_classes,transfer_loss='mmd'):
        super(create_model, self).__init__()
        self.device = device
        self.transfer_loss = transfer_loss
        Shared_layer_list = [nn.Linear(feature_Size, 512), 
                             nn.ReLU(),
                             nn.BatchNorm1d(512),
                             nn.Linear(512, 512),
                             nn.ReLU(),
                             nn.BatchNorm1d(512),
                             nn.Dropout(p=0.1),
                             nn.Linear(512, num_classes)]
        
        self.Shared_layer = nn.Sequential(*Shared_layer_list) 
        
        
    def forward(self, source, target):
               
        source_clf = self.Shared_layer(source)
        target_clf = self.Shared_layer(target)
        transfer_loss =  self.adapt_loss(source_clf, target_clf, self.transfer_loss)

        return source_clf, transfer_loss
    
        
    
    def predict(self, x):

        clf = self.Shared_layer(x)
        return clf

    def adapt_loss(self, X, Y, adapt_loss):

        if adapt_loss == 'mmd':
            mmd_loss = mmd.MMD_loss()
            loss = mmd_loss(X, Y) 
        else:
            loss = 0
        return loss     
    
        
    
    
def batch_generator(data, batch_size):
    """Generate batches of data.

    Given a list of numpy data, it iterates over the list and returns batches of the same size
    This
    """
    all_examples_indices = len(data[0])
    while True:
        mini_batch_indices = np.random.choice(all_examples_indices, size=batch_size, replace=False)
        tbr = [k[mini_batch_indices] for k in data]
        yield tbr    
        
    
    

cuda


In [2]:
start=datetime.now()
def set_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    
set_seed(99) 

feature_Size=2560
all_num_classes=[10,31,65]


datasets=['Office_caltech_10','Office31','Office_home'];
str_domains_Office_caltech_10 = ['caltech', 'amazon', 'webcam', 'dslr']; 
str_domains_Office31 = ['amazon', 'webcam', 'dslr']; 
str_domains_Office_home = ['Art', 'Clipart', 'Product', 'Real_World']; 

batch_size =64
epochs = 9
n_batch =30
T = 3
prob=[0.5,0.8,0.9]


    
Acc1={}
for k in range(len(datasets)-2):
    k+=1
    print('Domain adaptation on '+ datasets[k] +' dataset')
    task=1
    for ii in range(len(eval('str_domains_'+datasets[k]))):
        x=loadmat('./' +datasets[k] + '/' +eval('str_domains_'+datasets[k])[ii]+'_efficientnet.mat')  #source features
        Xs = torch.from_numpy(x['features']).to(device)
        Ys = torch.from_numpy(np.squeeze(x['labels'], axis=1)).long().to(device)
        S_batches = batch_generator([Xs, Ys], batch_size)
        for jj in range(len(eval('str_domains_'+datasets[k]))):
            if ii ==jj:
                 continue
            print(eval('str_domains_'+datasets[k])[ii] + ' to ' + eval('str_domains_'+datasets[k])[jj])

            xx=loadmat('./' +datasets[k] + '/' +eval('str_domains_'+datasets[k])[jj]+'_efficientnet.mat') #target features

            Xt = torch.from_numpy(xx['features']).to(device)
            Yt = torch.from_numpy(np.squeeze(xx['labels'], axis=1)).long().to(device)
            T_batches = batch_generator([Xt,Yt], batch_size)

            model=create_model(Xs.shape[1],all_num_classes[k],transfer_loss='mmd')
            model =model.to(device)

            criterion = nn.CrossEntropyLoss().to(device)
            optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

            Yt_p = None                
            for t in range(T+1):# Recurrent Pseudo-Labeling
                if Yt_p is not None:
                    Xs = torch.from_numpy(x['features']).to(device)
                    Ys = torch.from_numpy(np.squeeze(x['labels'], axis=1)).long().to(device) # To prevent bad labels
                    sm = torch.nn.Softmax()
                    probabilities = sm(Yt_p) 
                    Xs=torch.cat([Xs,Xt[probabilities.max(axis=1).values>prob[t-1],:]], 0)
                    Ys=torch.cat([Ys,pred[probabilities.max(axis=1).values>prob[t-1]]], 0)         
                    S_batches = batch_generator([Xs, Ys], batch_size)

                for epoch in range(epochs): 
                    for i in range(n_batch):

                        X0, Y0 = next(S_batches)
                        X1, Y1 = next(T_batches)   
                        running_loss = 0.0
                        optimizer.zero_grad()
                        outputs, transfer_loss= model(X0,X1)
                        clf_loss = criterion(outputs, Y0)   
                        loss = clf_loss + transfer_loss 
                        loss.backward()
                        optimizer.step()

                Yt_p = model.predict(Xt)
                pred = torch.max(Yt_p, 1)[1]
                Acc = 100 * (pred == Yt).sum().item() / Yt.size(0)
                print(round(Acc,2))   
                Acc1[t,task]=round(Acc,2)
            print(datetime.now()-start)
            Acc1[t,task]=round(Acc,2)
            task+=1       
            
all_mean=np.mean(np.array(list(Acc1.items()))[:,1].reshape((int(np.array(list(Acc1.items())).shape[0]/(T+1)), T+1)),0)
for t in range(T+1):
    if t==0:
        print('Average accuracy original:', round(all_mean[t],2)) 
    else:
        print('Average accuracy '+ str(t) +' self learning:', round(all_mean[t],2))
print(datetime.now()-start)             

Domain adaptation on Office31 dataset
amazon to webcam
92.08


C:\Users\zys61\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


95.35
95.6
95.85
0:00:09.986435
amazon to dslr
95.98
96.99
96.79
96.99
0:00:18.119342
webcam to amazon
80.37
81.82
82.22
82.43
0:00:26.337195
webcam to dslr
98.59
99.2
99.2
99.2
0:00:34.150993
dslr to amazon
80.05
82.14
82.82
82.96
0:00:42.270286
dslr to webcam
96.1
96.98
97.11
97.11
0:00:50.159086
Average accuracy original: 90.53
Average accuracy 1 self learning: 92.08
Average accuracy 2 self learning: 92.29
Average accuracy 3 self learning: 92.42
0:00:50.160083


C:\Users\zys61\Anaconda3\lib\site-packages\ipykernel_launcher.py:89: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
